In [106]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [107]:
#Import Train Dataset
df_train = pd.read_csv('train.csv')
df_train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


As per the previous EDA, I select Pclass, Sex, Embarked and Survived fields to predict the test set by using Decision Tree Algorithum. 
(EDA Link can be found on Titanic prediction by using KNN Algorithum - Link: https://github.com/ChinthakaAsiri/Data-Science/blob/master/Titanic%20Data%20Challange/Submission%20Version%201%20(Score%2071%25)/Titanic-ML-V1.ipynb )

In [108]:
#Import the test data set
df_test = pd.read_csv('test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [109]:
# Combine Train and Test sets. Then assign number to Sex and Embarked fields by using LabelEncorder. Then seperate the dataset again. 
df_all = df_train.append(df_test)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [110]:
from sklearn.preprocessing import LabelEncoder

In [111]:
le_Sex = LabelEncoder()
le_Embarked = LabelEncoder()

In [112]:
#Convert Sex and Embarked fields as string (presently they are object type)
df_all['Sex'] = df_all['Sex'].astype('string')
df_all['Embarked'] = df_all['Embarked'].astype('string')
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   string 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   string 
dtypes: float64(3), int64(4), object(3), string(2)
memory usage: 132.9+ KB


In [113]:
#Drop the missing value in the Embarked (Originally in Train Set)
df_all = df_all.dropna(axis = 'index', how = 'any', subset= ['Embarked'])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1307 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1307 non-null   int64  
 1   Survived     889 non-null    float64
 2   Pclass       1307 non-null   int64  
 3   Name         1307 non-null   object 
 4   Sex          1307 non-null   string 
 5   Age          1044 non-null   float64
 6   SibSp        1307 non-null   int64  
 7   Parch        1307 non-null   int64  
 8   Ticket       1307 non-null   object 
 9   Fare         1306 non-null   float64
 10  Cabin        293 non-null    object 
 11  Embarked     1307 non-null   string 
dtypes: float64(3), int64(4), object(3), string(2)
memory usage: 132.7+ KB


In [114]:
df_all['Sex_en'] = le_Sex.fit_transform(df_all['Sex'])
df_all['Embarked_en'] = le_Sex.fit_transform(df_all['Embarked'])
df_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_en,Embarked_en
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,2
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,2


In [115]:
#Break the dataframe back into Train and Test data
df_train.shape

(891, 12)

In [116]:
df_test.shape

(418, 11)

In [117]:
#New train dataset has 889 entries (Two columns were removed in here)
df_train_n = df_all.iloc[:889,:]
df_test_n = df_all.iloc[889:,:]
df_train_n.shape

(889, 14)

In [118]:
df_test_n.shape

(418, 14)

In [119]:
df_test_n.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_en,Embarked_en
0,892,NaN,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,1
1,893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,2
2,894,NaN,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,1
3,895,NaN,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,2
4,896,NaN,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,2


In [120]:
#Finalize the train set and the test set
df_train_inputs = df_train_n[['Sex_en','Embarked_en','Pclass']]
df_train_n['Survived'] = df_train_n['Survived'].astype('int')
df_train_target = df_train_n['Survived']
df_train_inputs.head()

C:\Users\Chinthaka Asiri\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Sex_en,Embarked_en,Pclass
0,1,2,3
1,0,0,1
2,0,2,3
3,0,2,1
4,1,2,3


In [121]:
df_test_inputs = df_test_n[['Sex_en','Embarked_en','Pclass']]
df_test_inputs.head()

,Sex_en,Embarked_en,Pclass
0,1,1,3
1,0,2,3
2,1,1,2
3,1,2,3
4,0,2,3


In [122]:
#Develop the ML Model
from sklearn import tree

In [124]:
model = tree.DecisionTreeClassifier()

In [125]:
model.fit(df_train_inputs, df_train_target)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [130]:
pred_array = model.predict(df_test_inputs)

In [131]:
df_test_output = pd.DataFrame(pred_array)
df_test_output.head()

,0
0,0
1,0
2,0
3,0
4,0


In [132]:
#Finalize the prediction list
df_pred = pd.concat([df_test_n,df_test_output], axis=1, join='inner')
df_pred.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_en,Embarked_en,0
0,892,NaN,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,1,0
1,893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,2,0
2,894,NaN,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,1,0
3,895,NaN,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,2,0
4,896,NaN,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,2,0


In [134]:
df_final_list = df_pred[['PassengerId', 0]]
df_final_list.columns = ['PassengerId', 'Survived']
df_final_list.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [136]:
df_final_list.to_csv('Chinthaka_Titanic_DT_V1.csv', index=0)

In [139]:
#2021-05-23 | 12:52AM (Chinthaka: https://github.com/ChinthakaAsiri)